In [1]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
import torch
import os
from utils import match_dates_based_on_precision, parse_date
import spacy
import json

/nas/home/amandeep/miniconda3/envs/python3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cache_dir='/data/amandeep/huggingface_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['XDG_CACHE_HOME'] = cache_dir
nlp = spacy.load('en_core_web_trf')

In [3]:
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq", cache_dir=cache_dir)

/nas/home/amandeep/miniconda3/envs/python3/lib/python3.9/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function i

In [4]:
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", 
                                         index_name="exact", 
                                         use_dummy_dataset=False, 
                                         cache_dir='/data/amandeep/huggingface_cache')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [5]:
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RagTokenForGeneration were not initialized from the model checkpoint at facebook/rag-token-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [ ]:
questions = ["When did Barack Obama get married?", 
             "When did Joe Biden get married?", 
             "When did Joe Biden and Neilia get married?",
             "When did Sachin Tendulkar get married?",
            "when did Kailin Curran marry Keanu?",
            "when did Guillaume Canet marry Diane?",
            "when did Sujarinee Vivacharawongse marry Vajiralongkorn I of?"]

In [ ]:
def run_rag(question):
    input_dict = tokenizer.prepare_seq2seq_batch(question, return_tensors="pt")
    # generated = model.generate(input_ids=input_dict["input_ids"], n_docs=100) 
    generated = model.generate(input_ids=input_dict["input_ids"]) 
    return tokenizer.batch_decode(generated, skip_special_tokens=True)

In [6]:
import json
from typing import List

In [ ]:
def answer_question(question: str) -> List[str]:
    inputs = tokenizer(question, return_tensors="pt")
    input_ids = inputs["input_ids"]

    question_hidden_states = model.question_encoder(input_ids)[0]
    docs_dict = retriever(input_ids.numpy(), question_hidden_states.detach().numpy(), return_tensors="pt")
    doc_scores = torch.bmm(
        question_hidden_states.unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2)
    ).squeeze(1)

    generated = model.generate(
        context_input_ids=docs_dict["context_input_ids"],
        context_attention_mask=docs_dict["context_attention_mask"],
        doc_scores=doc_scores,
    )
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)
    return generated_string

In [7]:
questions_f = open('../data/dpr-post-process-unified-qa-matched.jl')

In [ ]:
rag_answers = []
c = 1
for line in questions_f:
    if c % 1000 == 0:
        print(c)
    c += 1 
    j = json.loads(line.strip())
    rag_answer = run_rag(j['q'])
    j['rag_answer'] = {'raw_answer': rag_answer}
    rag_answers.append(j)

In [ ]:
rag_answers[7706]

In [8]:
q_to_p_dict = json.load(open('../data/questions_o_precision_dict.json'))

In [ ]:
for ra in rag_answers:
    rag_raw_dates = ra['rag_answer']['raw_answer']
    ra_matched = False
    for rrd in rag_raw_dates:
        parsed_dates = parse_date(rrd, nlp)
        for pd in parsed_dates:
            ra_matched, prov = match_dates_based_on_precision(ra['a'], q_to_p_dict[ra['q']], pd)
            if ra_matched:
                ra['rag_answer']['parsed_date'] = pd 
                break
        ra['rag_answer']['matched'] = ra_matched

In [ ]:
rag_answers[7706]

In [ ]:
c = 0
matched_rag_answers = []
for ra in rag_answers:
    rag_a = ra['rag_answer']
    if rag_a['matched']:
        c += 1
        matched_rag_answers.append(ra)
print(c)

In [ ]:
matched_rag_answers[10]

In [ ]:
o = open('../data/dpr-post-process-unified-qa-and-rag-matched.jl', 'w')
for mra in matched_rag_answers:
    o.write(json.dumps(mra))
    o.write('\n')
o.close()

In [ ]:
def run_rag_with_context(question, dpr_answer):
    input_dict = tokenizer.prepare_seq2seq_batch(question, tgt_texts = dpr_answer, return_tensors="pt")
    generated = model.generate(input_ids=input_dict["input_ids"], 
                               attention_mask=input_dict['attention_mask'],
                              context_input_ids=input_dict['labels'], n_docs=1) 
    return tokenizer.batch_decode(generated, skip_special_tokens=True)

In [ ]:
i_d = run_rag_with_context('when did Prince Christian of Hanover marry Alessandra de?',
                           'Following his father\'s marriage to Princess Caroline, Christian and his family moved to Fontainebleau, Paris, France. Christian later continued with his education at Malvern College. In 2004, Christian\'s father signed over to his sons the German property of the House of Hanover, including Marienburg Castle. The two brothers hired a Sotheby\'s team to auction off some of the castle\'s content in order to save its finances. On 24 November 2017, Christian married Peruvian lawyer Alessandra de Osma Foy (born 1988), daughter of Felipe de Osma Berckemeyer and Elizabeth Foy Vásquez, during a civil service at the Chelsea and Westminster register')
                           

In [9]:
def rag_token_generate_batch(questions, tokenizer, retriever, model, torch_device='cpu'):
    
    input_dict = tokenizer(
        questions,
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    input_ids = input_dict.input_ids.to(torch_device)
    attention_mask = input_dict.attention_mask.to(torch_device)
    output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        n_docs=100
    )
    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return outputs

In [10]:
rag_token_generate_batch(['when did Sachin Tendulkar marry Anjali?'], tokenizer, retriever, model)

/nas/home/amandeep/miniconda3/envs/python3/lib/python3.9/site-packages/transformers/generation_utils.py:2129: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


[' 24 august 1971']

In [16]:
questions_f = open('../data/dpr-post-process-unified-qa-matched.jl')
questions = []
for line in questions_f:
    questions.append(json.loads(line.strip()))
questions_f.close()

In [18]:
rag_o = open('../data/dpr-post-process-unified-qa-plus-rag-k-100.jl', 'w')

In [ ]:
c = 0
for q in questions:
    if c%1000 == 0:
        print(c)
    c += 1
    q['rag_answer'] = rag_token_generate_batch(q['q'], tokenizer, retriever, model)
    rag_o.write(json.dumps(q))
    rag_o.write('\n')

0


/nas/home/amandeep/miniconda3/envs/python3/lib/python3.9/site-packages/transformers/generation_utils.py:2129: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1000
2000
3000
4000
5000
6000


In [17]:
rag_o.close()